In [ ]:
import re
from tkinter import Tk
from collections import defaultdict

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as mpatches

%matplotlib inline

In [ ]:
# lazy solution for getting Java output: get via clipboard

# TODO: this does not close TK cleanly 
# how to close the tk app background process?
t = Tk()
clip = t.clipboard_get()

In [ ]:
# parse the output of registration accuracy measurements

group_idx = -1
res = defaultdict(list)

p = re.compile('tpId=0 setupId=([0-9]+?), mean error : (.*)')
for line in clip.split('\n'):
    if line.startswith('0-'):
        group_idx += 1
    m = p.match(line)
    if m:
        err = float(m.groups()[1])
        res[group_idx].append(err)
        
print(res)

np.mean([np.mean(v) for k,v in res.items()])

In [ ]:
# parse the output of maximum registration accuracy measurements
# (using just manual beads)

mns = []
p = re.compile('.*?Avg Error: (.*?)px')

for line in clip.split('\n'):
    m = p.match(line)
    if m:
        #print(m.groups())
        err = float(m.groups()[0])
        mns.append(err)

print(np.mean(mns))

In [ ]:
# values collected from the runs above

means = {
    'nr-angle-illum' : 2.671840472198133,
    'nr-angle' : 3.6628449365984137,
    'nr-all' : 6.2457957937289805,
    'icp-all' : 8.052441664829841,
    'icp-angle' : 4.722972984706986,
    'icp-angle-illum' : 5.757628831721657,
    'tr-all' : 11.521826004374818,
    'tr-angle' : 7.331319951725959,
    'tr-angle-illum' : 7.586251326558656,
}

theor_min = {
    'nr-angle-illum' : 0.05851794422411804,
    'nr-angle' : 0.09027884418293469,
    'nr-all' : 0.15411947574067628,
    'tr-angle-illum' : 2.9203602846828463,
    'tr-angle' : 4.7476080455799305,
    'tr-all' : 8.999987265179568,
    'icp-angle-illum': 2.38294535936038,
    'icp-angle' : 3.973140371328637,
    'icp-all' : 5.960762898016536   
}

In [ ]:
# create plot

cols = ['lightgrey'] * 3 + ['darkgrey'] * 3 + ['grey'] * 3
xticks = (
    list(reversed(['tr-angle-illum', 'tr-angle', 'tr-all'])) +
    list(reversed(['icp-angle-illum', 'icp-angle', 'icp-all'])) +
    list(reversed(['nr-angle-illum', 'nr-angle', 'nr-all']))
)

plt.figure(figsize=(12,8))
for idx, (m, t, c) in enumerate(zip([means[k] for k in xticks],[theor_min[k] for k in xticks], cols)):
    plt.bar(2*idx-0.4, m, color=c, edgecolor='black', linewidth=2,)
    plt.bar(2*idx+0.4, t, color=c, edgecolor='black', linewidth=2, linestyle='dashed')
    
plt.xticks(np.arange(0,2*len(xticks),2), xticks)
plt.ylabel('mean error (px)')
plt.title('BigStitcher registration accuracy')

legend_patches = [
    mpatches.Rectangle([0,0], 0, 0, facecolor='lightgrey', edgecolor='black',
                               linewidth=2, label='Translation, real errors'),
    mpatches.Rectangle([0,0], 0, 0, facecolor='lightgrey', edgecolor='black',
                               linewidth=2, linestyle='dashed', label='Translation, theoretical min. errors'),
    mpatches.Rectangle([0,0], 0, 0, facecolor='darkgray', edgecolor='black',
                               linewidth=2, label='Affine, real errors'),
    mpatches.Rectangle([0,0], 0, 0, facecolor='darkgray', edgecolor='black',
                               linewidth=2, linestyle='dashed', label='Affine, theoretical min. errors'),
    mpatches.Rectangle([0,0], 0, 0, facecolor='gray', edgecolor='black',
                               linewidth=2, label='Non-Rigid, real errors'),
    mpatches.Rectangle([0,0], 0, 0, facecolor='gray', edgecolor='black',
                               linewidth=2, linestyle='dashed', label='Non-Rigid, theoretical min. errors'),
]

plt.legend(handles=legend_patches)
